In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Tracked_Profiles'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#################################
#LOADING DATA

In [ ]:
#NEEDED TO PLOT THE CORRECT DATA
data_type="Tracked_Area"

In [ ]:
type1='CL';type2='nonCL'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)
        # print(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
    # print(key)

In [ ]:
type1='SBZ';type2='nonSBZ'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
    # print(key)

In [ ]:
############################
#PLOTTING

In [ ]:
#LIMITING Y AXIS
limit_y=True
limit_y=False

In [ ]:
def limit_axes_to_y(ax, y_min=0, y_max=7, buffer_frac=0.1):
    ax.set_ylim(y_min, y_max)

    x_limited = []
    for line in ax.get_lines():
        xdata, ydata = np.array(line.get_xdata()), np.array(line.get_ydata())
        # Mask for y within limits
        y_mask = (ydata >= y_min) & (ydata <= y_max)
        # Apply mask
        x_visible = xdata[y_mask]
        # Remove NaN or Inf from x_visible
        x_visible = x_visible[np.isfinite(x_visible)]
        x_limited.extend(x_visible)
    if len(x_limited) > 0:
        x_limited = np.array(x_limited)
        x_min, x_max = np.min(x_limited), np.max(x_limited)

        if not (np.isfinite(x_min) and np.isfinite(x_max)):
            print("Warning: Non-finite x-limits detected, skipping set_xlim")
            return
        x_range = x_max - x_min

        if x_range == 0:
            buffer = 0.1  # fixed small buffer
        else:
            buffer = buffer_frac * x_range
        ax.set_xlim(x_min - buffer, x_max + buffer)
    else:
        print("Warning: No visible x data within y limits to set xlim")


In [ ]:
def averaged_profiles(profile): 
    out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data
    out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
    return out_var

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
all_cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {all_cloudbase}\n")

In [ ]:
def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
def PlotTrackedAreaProfile(types, type3, linestyles, color, ax=None):
    lw = 1.25

    # Create new figure and axes if none provided
    if ax is None:
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax = np.ravel(ax)  # Ensure ax is flat list/array

    factor = 1
    switch = 1

    # Build labels for each type
    profile_labels = {
        t: {
            'label': f"{type3} ({t})",
            'linestyle': linestyles[i],
            'color': color,
            'alpha': 0.15,
            'hatch': None  # Or customize per type
        }
        for i, t in enumerate(types)
    }

    profile_types = ['general', 'cloudy']

    for i, prof_type in enumerate(profile_types):
        for t in types:
            array_name = f"{t}_{type3}_profile_array_{prof_type}"
            SE_name = f"{t}_{type3}_profile_array_{prof_type}_SE"

            if array_name not in globals() or SE_name not in globals():
                print(f"Skipping {t} {type3} {prof_type} – missing data.")
                continue

            profile_array = globals()[array_name]
            SE_array = globals()[SE_name]

            profile = averaged_profiles(profile_array)
            SE_profile = SE_array

            ax[i].plot(profile[:, 0], profile[:, 1],
                       color=profile_labels[t]['color'],
                       linestyle=profile_labels[t]['linestyle'],
                       label=profile_labels[t]['label'],
                       lw=lw,
                       zorder=10)

            fill_kwargs = {
                'alpha': profile_labels[t]['alpha'],
                'linewidth': 0.0,
                'color': profile_labels[t]['color']
            }

            ax[i].fill_betweenx(
                profile[:, 1],
                np.maximum(profile[:, 0] - factor * SE_profile[:, 0] * switch, 0),
                profile[:, 0] + factor * SE_profile[:, 0] * switch,
                **fill_kwargs
            )

        ax[i].grid(True)
        ax[i].set_xlabel(r'A ($km^2$)')
        ax[i].set_ylabel('z (km)')
        ax[i].axhline(all_cloudbase, color='purple', linestyle='dashed')
        ax[i].axhline(MeanLFC / 1000, color='green', linestyle='dashed')
        ax[i].set_title(f'{prof_type.capitalize()} Profile')

    if ax is None:
        # fix_x_limits(ax)
        # fix_y_limits(ax)
        plt.suptitle(f'{type3} {" vs ".join(types)} Updrafts Area Profile')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])


In [ ]:
def SavePlot(fig, type1, res, t_res, Np_str, dir2, limit_y=False):
    # Define output directory
    subdir_name = f'{res}_{t_res}_{Np_str}'
    output_dir = os.path.join(dir2, 'Project_Algorithms', 'Tracked_Profiles', 'PLOTS', 'Tracked_Area', subdir_name)

    if limit_y==True:
        output_dir = os.path.join(output_dir, "SLICE")
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    #SAVE PATH
    filename = f'{type1}_Area_Tracked_Profiles_{res}_{t_res}_{Np_str}.jpg'
    save_path = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Saved figure to: {save_path}")


In [ ]:
#######################
#CL vs nonCL
#SBZ vs nonSBZ

In [ ]:
############################
#PLOTTING

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns
ax = ax.reshape(1, 2)  # Ensure predictable indexing
linestyles = ['solid', 'dashed', 'dashdot', 'dotted']

# ===== First group: CL vs nonCL =====
PlotTrackedAreaProfile(['CL', 'nonCL'], 'ALL', [linestyles[0], linestyles[1]], color='black', ax=ax[0])
PlotTrackedAreaProfile(['CL', 'nonCL'], 'SHALLOW', [linestyles[0], linestyles[1]], color='green', ax=ax[0])
PlotTrackedAreaProfile(['CL', 'nonCL'], 'DEEP', [linestyles[0], linestyles[1]], color='blue', ax=ax[0])

# ===== Second group: SBZ only (no nonSBZ) =====
PlotTrackedAreaProfile(['SBZ'], 'ALL', [linestyles[2]], color='black', ax=ax[0])
PlotTrackedAreaProfile(['SBZ'], 'SHALLOW', [linestyles[2]], color='green', ax=ax[0])
PlotTrackedAreaProfile(['SBZ'], 'DEEP', [linestyles[2]], color='blue', ax=ax[0])

# ===== Common formatting =====
for axis_group in ax:
    for axis in axis_group:
        if limit_y:
            limit_axes_to_y(axis, y_min=0, y_max=7)
        axis.set_xlim(left=0)
        axis.set_ylim(bottom=0)

# Optional: apply fixes to whole array
# fix_x_limits(ax.flatten())
# fix_y_limits(ax.flatten())

# ===== Titles =====
ax[0, 0].set_title('General Updraft')
ax[0, 1].set_title('Cloudy Updraft')

# # ===== FIX TICKS =====
#FIXING TICKS
axises=[ax[0,0],ax[0,1]]
SnapLimitsToTicks(axises, dim='x')
MatchAxisLimits(axises,dim='x')
if limit_y==False:
    axes=fig.get_axes()
    for axis in axes:
        axis.set_ylim(bottom=0,top=20)
# apply_scientific_notation(axises,decimals=2,scientific=True)

# ===== Global title and layout =====
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# ===== Legend: Types (linestyles) =====
custom_lines = [
    Line2D([0], [0], color='black', linestyle='solid', linewidth=1.5, label='CL'),
    Line2D([0], [0], color='black', linestyle='dashed', linewidth=1.5, label='nonCL'),
    Line2D([0], [0], color='black', linestyle='dashdot', linewidth=1.5, label='SBF'),
    # No nonSBZ line — it’s not plotted
]

fig.legend(
    handles=custom_lines,
    loc='upper center',
    ncol=4,
    fontsize=10,
    title='Types',
    title_fontsize=12,
    bbox_to_anchor=(0.5, 1.03),
    borderaxespad=0,
    frameon=True
)

# ===== Legend: Colors for cloud depth =====
color_legend_lines = [
    Line2D([0], [0], color='black', linestyle='solid', linewidth=2, label='ALL'),
    Line2D([0], [0], color='green', linestyle='solid', linewidth=2, label='SHALLOW'),
    Line2D([0], [0], color='blue', linestyle='solid', linewidth=2, label='DEEP'),
]

ax[0, 1].legend(
    handles=color_legend_lines,
    loc='upper right',
    title='Cloud Types',
    title_fontsize=10,
    fontsize=9,
    frameon=True
)

# ===== Save Figure =====
SavePlot(fig, 'CLnonCL_SBF', res, t_res, Np_str, dir2, limit_y)


In [ ]:
limit_y=True